In [1]:
#PART1
#You will need to manually amend the target datasets for this script

library(bigrquery)

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

# Amend these to "point" at the datasets you want to query

targetdb1 <-'yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8'
targetdb1 <-gsub(' ','',targetdb1)
print (targetdb1)



targetdb2 <-'yhcr-prd-phm-bia-core.CB_MYSPACE_JDB'
targetdb2 <-gsub(' ','',targetdb2)
print (targetdb2)



[1] "yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8"
[1] "yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"


In [4]:
#PART2
# Firstly use the person table in the FDM to find the person_id'd
# Open HowToBuildACohort_BasedonAge.sql  in Bigquery to show 
# in BigQuery interface


sql3 <-paste('SELECT person_id
, birth_datetime 
, CURRENT_DATE() as todaysDate
,DATE_DIFF(CURRENT_DATE(),birth_datetime,YEAR) as currage
,EXTRACT(YEAR FROM birth_datetime ) AS yearborn
FROM ',targetdb1,'.person b 
where DATE_DIFF(CURRENT_DATE(),birth_datetime,YEAR) >= 64 
and DATE_DIFF(CURRENT_DATE(),birth_datetime,YEAR) <=95
and death_datetime is not null ;', sep = "")
#This runs it
tb3 <- bq_project_query(project_id, sql3)

#This loads  it into an R data frame
table <- bq_table_download(tb3)
#This displays it
#table

In [2]:
#PART3
# Using the query above create a temporary table in MYSPACE
# with just these person_id's

sql4 <-paste('create or replace table ',targetdb2,'.tmp_cohort
as 
SELECT person_id
FROM ',targetdb1,'.person b 
where DATE_DIFF(CURRENT_DATE(),birth_datetime,YEAR) >= 64 
and DATE_DIFF(CURRENT_DATE(),birth_datetime,YEAR) <= 95
and death_datetime is not null ;', sep = "")
#This runs it
tb4 <- bq_project_query(project_id, sql4)

#This loads  it into an R data frame
table <- bq_table_download(tb4)
#This displays it
#table

In [3]:
#PART4
#Finally extract the coded data for this cohort into a
# data frame 
#then into another temporary table in MYSPACE


sql5 <-paste('select distinct a.person_id, 
a.ctv3code,
 from ',targetdb1,'.tbl_srcode a, ',targetdb2,'.tmp_cohort b 
where a.person_id = b.person_id ', sep = "")
#This runs it
tb5 <- bq_project_query(project_id, sql5)

#This loads  it into an R data frame
table <- bq_table_download(tb5)
#This displays it
table



person_id,ctv3code
<int>,<chr>
10445703,2469.
1051284,42L..
12798226,X74UV
13014612,X77Wi
13324090,Y0529
13610366,Xa05Q
12769142,XE2m6
13153530,42P..
13207472,XM1T1


In [21]:
#PART5
# I can use two ways of writing this datafram back to MYSPPACE

# either use a create table sql script  or write the dataframe produced
# in Part4 (above) and write this back as a table.

#So creating a table

sql5 <-paste('create table ',targetdb2,'.tmp_mydata 
as select distinct a.person_id, 
a.ctv3code,
 from ',targetdb1,'.tbl_srcode a, ',targetdb2,'.tmp_cohort b 
where a.person_id = b.person_id ', sep = "")
#This runs it
tb5 <- bq_project_query(project_id, sql5)

#essentially this reruns the code above 

In [23]:
#PART6
#or 

#Ensure Part4 has been run or we will have no dataframe called "table"

#Now we will write back the data we got from our dataframe  above into a table in out MYSPACE
#called tmp_mydata2 to show the difference

mybq = bq_table(project='yhcr-prd-phm-bia-core', dataset='CB_MYSPACE_JDB', table='tmp_mydata2')
bq_table_upload(x=mybq, values=table, create_disposition='CREATE_IF_NEEDED', 
                write_disposition='WRITE_APPEND')